In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np
import pandas as pd
from os.path import join
from glob import glob
from TorchEASE.src.main.EASE import TorchEASE
import time
import torch

In [3]:
data_path = '../data/msd/pro_sg'

In [4]:
train_df = pd.read_csv(join(data_path, f"train.csv"))

val_df_tr = pd.read_csv(join(data_path, f"validation_tr.csv"))
val_df_te = pd.read_csv(join(data_path, f"validation_te.csv"))

test_df_tr = pd.read_csv(join(data_path, f"test_tr.csv"))
test_df_te = pd.read_csv(join(data_path, f"test_te.csv"))

In [5]:
# train_df = train_df[:int(1e5)]

In [7]:
train_df.head()

,uid,sid
0,44118,0
1,44118,1
2,44118,2
3,44118,3
4,44118,4


In [5]:
te_implicit = TorchEASE(train_df, user_col="uid", item_col="sid")

2021-06-30 10:59:37 [INFO] notebook - Building user + item lookup
2021-06-30 10:59:38 [INFO] notebook - Building item hashmap
2021-06-30 10:59:46 [INFO] notebook - User + item lookup complete
2021-06-30 10:59:47 [INFO] notebook - Sparse data built


In [6]:
t0 = time.time()
te_implicit.fit()
print(time.time() - t0)

2021-06-30 10:59:47 [INFO] notebook - Building G Matrix
2021-06-30 11:57:28 [INFO] notebook - Building B matrix
3468.7566916942596


In [10]:
t0 = time.time()
a=evaluate(te_implicit, val_df_tr, val_df_te, k=20)
print(time.time() - t0)

KeyboardInterrupt: 

In [ ]:
def ndcg_at(predictions, labels, k=10, assume_unique=True):
    """Compute the normalized discounted cumulative gain at K.
    Compute the average NDCG value of all the queries, truncated at ranking
    position k. The discounted cumulative gain at position k is computed as:
        sum,,i=1,,^k^ (2^{relevance of ''i''th item}^ - 1) / log(i + 1)
    and the NDCG is obtained by dividing the DCG value on the ground truth set.
    In the current implementation, the relevance value is binary.
    If a query has an empty ground truth set, zero will be used as
    NDCG together with a warning.
    Parameters
    ----------
    predictions : array-like, shape=(n_predictions,)
        The prediction array. The items that were predicted, in descending
        order of relevance.
    labels : array-like, shape=(n_ratings,)
        The labels (positively-rated items).
    k : int, optional (default=10)
        The rank at which to measure the NDCG.
    assume_unique : bool, optional (default=True)
        Whether to assume the items in the labels and predictions are each
        unique. That is, the same item is not predicted multiple times or
        rated multiple times.
    Examples
    --------
    >>> # predictions for 3 users
    >>> preds = [[1, 6, 2, 7, 8, 3, 9, 10, 4, 5],
    ...          [4, 1, 5, 6, 2, 7, 3, 8, 9, 10],
    ...          [1, 2, 3, 4, 5]]
    >>> # labels for the 3 users
    >>> labels = [[1, 2, 3, 4, 5], [1, 2, 3], []]
    >>> ndcg_at(preds, labels, 3)
    0.3333333432674408
    >>> ndcg_at(preds, labels, 10)
    0.48791273434956867
    References
    ----------
    .. [1] K. Jarvelin and J. Kekalainen, "IR evaluation methods for
           retrieving highly relevant documents."
    """
    # validate K
    _require_positive_k(k)

    def _inner_ndcg(pred, lab):
        if lab.shape[0]:
            # if we do NOT assume uniqueness, the set is a bit different here
            if not assume_unique:
                lab = np.unique(lab)

            n_lab = lab.shape[0]
            n_pred = pred.shape[0]
            n = min(max(n_pred, n_lab), k)  # min(min(p, l), k)?

            # similar to mean_avg_prcsn, we need an arange, but this time +2
            # since python is zero-indexed, and the denom typically needs +1.
            # Also need the log base2...
            arange = np.arange(n, dtype=np.float32)  # length n

            # since we are only interested in the arange up to n_pred, truncate
            # if necessary
            arange = arange[:n_pred]
            denom = np.log2(arange + 2.)  # length n
            gains = 1. / denom  # length n

            # compute the gains where the prediction is present in the labels
            dcg_mask = np.in1d(pred[:n], lab, assume_unique=assume_unique)
            dcg = gains[dcg_mask].sum()

            # the max DCG is sum of gains where the index < the label set size
            max_dcg = gains[arange < n_lab].sum()
            return dcg / max_dcg

        else:
            return _warn_for_empty_labels()

    return _mean_ranking_metric(predictions, labels, _inner_ndcg)

In [48]:
unique_val_users = val_df_tr.uid.unique()

In [49]:
curr_idxes = val_df_tr.uid.isin(unique_val_users[:1])
play_val = val_df_tr[curr_idxes]

In [50]:
t0 = time.time()
predictions = te_implicit.predict(play_val, k=100)
print(time.time() - t0)

0.18633031845092773


In [51]:
uid_to_prediction = dict(zip(play_val.uid.unique(), predictions.numpy()))

In [52]:
a = val_df_te.uid.isin(unique_val_users[:1])
play_val_te = val_df_te[a]

In [53]:
preds_with_true = play_val_te.groupby("uid").agg({'sid':list})
preds_with_true = preds_with_true.reset_index()

In [54]:
preds_with_true['preds'] = preds_with_true.apply(lambda x:uid_to_prediction.get(x.uid), axis=1)

In [55]:
preds_with_true

,uid,sid,preds
0,512467,"[195, 18094, 13873, 1519, 1972, 5705, 7414, 74...","[2199, 103, 351, 1519, 223, 1534, 2751, 2984, ..."


In [59]:
preds_with_true.apply(lambda x:recall_at_k(x.sid, x.preds, 50), axis=1).mean()

0.3333333333333333

In [62]:
a = val_df_te[val_df_te.uid == unique_val_users[0]].groupby('uid')